First, load the data,in this notebook we are using the smokoy mountain data as the example. And will use basic AE to get the transformation of the data and do prediction using the same models.

In [1]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

2025-01-13 16:20:33.700593: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-13 16:20:33.703150: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 16:20:33.961652: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 16:20:34.024252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736803234.214208  626068 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736803234.22

In [2]:
import pandas as pd
import numpy as np
import random
from sklearn import svm
import pickle
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available:  0


2025-01-13 16:20:36.737125: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### 1. Load the data and data preprocess
After reading the data, convert the abundance matrix into relative abundance matrix; for the response variables, since we have multiple variables, organize them into an array with each variable as a column.

In [4]:
data,burn_label,un_label,duration_label,ASVs,df=loadData.loadSoilData("../data/count_table/merge_proportion_asv_smoky_moutain.csv")

In [5]:
data = FS.relative_abundance(data)

### 2.AE


In [6]:
data_train, data_test, y_train, y_test = RunML_continue.split_and_scale_data(data,burn_label)
print(data_train.shape)
print(data_test.shape)

(41, 1768)
(18, 1768)


In [7]:
AE_train, AE_test = RunML_continue.run_AE(data_train,data_test)

latent dimension: 100, activation:tanh, opt:adam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [8]:
data_AE = pd.concat([AE_train, AE_test], axis=0).to_numpy()

In [9]:
iter =30
cls = ["RF","SVM", "CatBoost","NB"]

In [10]:
data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }

In [11]:
print(np.shape(data))
print(np.shape(data_AE))

(59, 1768)
(59, 100)


The  function will print out the accuracy and AUC for each dataset using each classifier, and also will return the y_actual, y_predict, y_predprob for future use.

In [12]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

TBB Warning: The number of workers is currently limited to 0. The request for 47 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



             RF_Accuracy    RF_AUC  SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures     0.780303  0.922917      0.746970  0.843750           0.845455   
AE              0.645455  0.569097      0.592424  0.588889           0.609091   

             CatBoost_AUC  NB_Accuracy    NB_AUC  
AllFeatures      0.887037     0.863636  0.801389  
AE               0.586111     0.674242  0.601157  


In [13]:
print(metric.metric_sum(dict_cm))

                      Accuracy  Precision    Recall  Specification       Mcc
AllFeatures_RF        0.779661   1.000000  0.235294       1.000000  0.423886
AllFeatures_SVM       0.745763   0.571429  0.470588       0.857143  0.348896
AllFeatures_CatBoost  0.847458   1.000000  0.470588       1.000000  0.622530
AllFeatures_NB        0.864407   0.846154  0.647059       0.952381  0.655004
AE_RF                 0.644068   0.300000  0.176471       0.833333  0.011835
AE_SVM                0.593220   0.333333  0.411765       0.666667  0.074189
AE_CatBoost           0.610169   0.350000  0.411765       0.690476  0.097820
AE_NB                 0.677966   0.333333  0.117647       0.904762  0.033578


In [14]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy    RF_AUC  SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures     0.556061  0.702698      0.556061  0.424444           0.524242   
AE              0.442424  0.409365      0.356061  0.715238           0.440909   

             CatBoost_AUC  NB_Accuracy    NB_AUC  
AllFeatures      0.582540     0.610606  0.605000  
AE               0.375873     0.506061  0.440317  


In [15]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy    RF_AUC  SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures     0.590909  0.457857      0.507576  0.471607           0.542424   
AE              0.454545  0.441250      0.527273  0.426964           0.424242   

             CatBoost_AUC  NB_Accuracy    NB_AUC  
AllFeatures      0.436429     0.507576  0.516071  
AE               0.415179     0.490909  0.494107  


####  Model 2

##### Annual/ Perennial
For comparison, only use the selected features for Annual/ Perennial classification. 

Even though the performance of our method is still not as good as Lasso, but it gives us better prediction using only 19 features compared with the union selection which has 86 featues.

In [16]:
indices_3 = np.where(weights[2,:] > stats.chi2.ppf(1 - 0.1, 1))[0]

NameError: name 'weights' is not defined

In [ ]:
indices_3_sort = sorted(indices_3, key=lambda col: weights[2,col], reverse=True)

In [ ]:
len(indices_3_sort)

In [ ]:
X_FS_dur = data[:,indices_3_sort]

In [ ]:
data_subset2 = {"AllFeatures":data, 
               "SelectMicro": X_FS_dur,
               "Lasso":X_lasso_dur,
               "Lasso_finetune":X_lasso_ft_dur,
               "Random":data
              }
print(np.shape(data))
print(np.shape(X_FS_dur))
print(np.shape(X_lasso_dur))
print(np.shape(X_lasso_ft_dur))

In [ ]:
dict_cm_dur2 = RunML_continue.runClassifier_FScompare(data_subsets= data_subset2,y= duration_label,N=iter,classifiers=cls,SMOTE=True)

##### burn
Let's also try it on burn label (expected to have better performance than Lasso, but ......)

So, we check the feature ratio for burn label.

In [ ]:
weights[0,:]
indices_1 = np.where(weights[0,:] > stats.chi2.ppf(1 - 0.1, 1))[0]
indices_1_sort = sorted(indices_1, key=lambda col: weights[0,col], reverse=True)
len(indices_1_sort)

X_FS_burn = data[:,indices_1_sort]
X_lasso = data[:,xlabel_lasso_burn]

In [ ]:
data_subset3 = {"AllFeatures":data, 
               "SelectMicro": X_FS_burn,
               "Lasso":X_lasso_burn,
               "Lasso_finetune":X_lasso_ft_burn,
               "Random":data
              }
print(np.shape(data))
print(np.shape(X_FS_burn))
print(np.shape(X_lasso))

In [ ]:
dict_cm_burn2 = RunML_continue.runClassifier_FScompare(data_subsets= data_subset3,y= burn_label,N=iter,classifiers=cls,SMOTE=True)

In [ ]:
targetLabel=burn_label
print(set(targetLabel))
selectedASV1 = [ASVs[i] for i in indices_1_sort]
selectedASV1_lasso = [ASVs[i] for i in xlabel_lasso_burn]
FS.plotPresenseRatio(X_FS_burn,targetLabel,selectedASV1,posLabel="Yes",posText="Burned",negText="Not Burned",entries=len(selectedASV1))

In [ ]:
FS.plotPresenseRatio(X_lasso,targetLabel,selectedASV1_lasso,posLabel="Yes",posText="Burned",negText="Not Burned",entries=len(selectedASV1_lasso))


In [ ]:
print(np.asarray(selectedASV1))
print(np.asarray(selectedASV1_lasso))
print(np.isin(selectedASV1_lasso, selectedASV1))

#### Model3
To compare, also consider the union of the features selected by Lasso: the result shows that
1. for burn label, lasso's RF performance is worse, SVM is much better
2. for un_label,lasso's RF-accuracy SVM-AUC are better, RF-AUC and SVM-accuracy are worse.
3. for dur label, lasso's RF performance is worse, SVM-accuracy is worse SVM-AUC is  better

In [ ]:
# select the union of lasso feature selection
xlabel_lasso_union = pd.concat([pd.Series(xlabel_lasso), pd.Series(xlabel_lasso_burn), pd.Series(xlabel_lasso_dur)]).drop_duplicates().reset_index(drop=True)
len(xlabel_lasso_union)

In [ ]:
X_lasso_union = data[:,xlabel_lasso_union]
X_lasso_union.shape

In [ ]:
data_subset = {
    #"AllFeatures":data, 
            #   "SelectMicro": X_FS,
               "Lasso":X_lasso_union
              #, "Random":data
              }

print(np.shape(X_lasso_union))

In [ ]:
for i in range(yList.shape[1]):
    targetLabel=yList[:,i]
    dict_cm_lasso = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls)
    for dataset_name, classifiers in dict_cm_lasso.items():
        for classifier_name, labels in classifiers.items():
            actual_labels = labels[0]
            predicted_labels = labels[1]
            metric.plot_confusion_matrices(actual_labels, predicted_labels,f"{dataset_name} - {classifier_name}")

### SHARP value
Compare the SHARP value of the significant features selected by our method and Lasso.(pending)

In [ ]:
#df_X_FS = pd.DataFrame(X_FS,columns=[ASVs[i] for i in selectedOTU_index] )
#df_X_FS.columns

In [ ]:
#RunML_continue.sharp_value(df_X_FS,burn_label,classifier_name)

In [ ]:
#df_X_lasso = pd.DataFrame(X_lasso_union,columns=[ASVs[i] for i in xlabel_lasso_union] )
#RunML_continue.sharp_value(df_X_lasso,burn_label,classifier_name)

### compare the first 15 index by their present ratio

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

entries=15
selectedOTU_index_15=selectedOTU_index[:entries]
selectedASVs_15=[ASVs[i] for i in selectedOTU_index_15]

X_FS_15=data[:,selectedOTU_index_15]


targetLabel=burn_label
print(set(targetLabel))
FS.plotPresenseRatio(X_FS_15,burn_label,selectedASVs_15,posLabel="Yes",posText="Burned",negText="Not Burned")

targetLabel=un_label
print(set(targetLabel))
FS.plotPresenseRatio(X_FS_15,un_label,selectedASVs_15,posLabel="Natural",posText="Natural",negText="Urban")


targetLabel=duration_label
print(set(targetLabel))
FS.plotPresenseRatio(X_FS_15,duration_label,selectedASVs_15,posLabel="Annual",posText="Annual",negText="Perennial")

In [ ]:
selectedASVs=[ASVs[i] for i in selectedOTU_index]

In [ ]:
print(np.asarray(selectedASVs))
print(len(selectedASVs))

### Negative Gini Impurity
Gini Impurity is the probability of incorrectly classifying a randomly chosen element in the dataset if it were randomly labeled according to the class distribution in the dataset. It’s calculated as:

$G = 1- \sum_{i=1}^C p_i^2$

where C is the number of classes. (which means it can be used to measure for multiple level classification)

Here I will use the negative Gini Impurity to measure each OTU, if NG is large (1) which means the OTU only exist in one class, if NG value is small($1/c$) which means the OTU is evenly distributed among  the classes.

$NG = \sum_{i=1}^C p_i^2$

In [ ]:
# NG for selected OTU
NG_selected = metric.Neg_GINI(X_FS,yList)
print(NG_selected.shape)
# NG for Not selected OTU
X_FS_none = np.delete(data, selectedOTU_index, axis=1)
NG_noselected = metric.Neg_GINI(X_FS_none,yList)
print(NG_noselected.shape)

In [ ]:
label_lasso = [xlabel_lasso_burn,xlabel_lasso,xlabel_lasso_dur]

In [ ]:
ng_lasso_list = []
for i in range(len(y_index)):
    X_lasso_ng = data[:,label_lasso[i]]
    print(X_lasso_ng.shape)
    X_lasso_none_ng = np.delete(data, label_lasso[i], axis=1)
    print(X_lasso_none_ng.shape)
    Ng1 = metric.Neg_GINI(X_lasso_ng,np.transpose(yList)[i])
    Ng2 = metric.Neg_GINI(X_lasso_none_ng,np.transpose(yList)[i])
    ng_lasso_list.append([Ng1,Ng2])


#### Compare lasso and SelectMicro

In [ ]:
# Number of subplots
num_plots = len(y_index)

# Create a figure with a grid of subplots
plt.figure(figsize=(4, 4 * num_plots))

# Loop through each index and create a subplot
for i in range(num_plots):
    plt.subplot(num_plots, 1, i + 1)  # (nrows, ncols, index)
    plt.boxplot([NG_selected[i, :], ng_lasso_list[i][0]], tick_labels=['SelectMicro', 'Lasso'])
    plt.title(f'NG results of the selected OTU vs. non-selected OTUs - {y_index[i]}')
    plt.ylabel('NG')
    plt.grid(axis='y')
    #print(len(NG_selected[i, :]))
# Adjust layout
plt.tight_layout()  # Adjusts the subplots to fit into the figure area.
plt.show()  # Show all plots at once

In [ ]:
# compare the selected and non select by our method
for i in range(len(y_index)):
    plt.figure(figsize=(4, 4))
    plt.boxplot([NG_selected[i,:], NG_noselected[i,:]], tick_labels=['SelectMicro', 'Not selected'])
    plt.title(f'NG results of the selected OTU vs. non selected OTUs by our method - {y_index[i]}')
    plt.ylabel('NG')
    plt.grid(axis='y')
    
# Show the plot
plt.show()

In [ ]:
# compare the selected and non select by lasso
# Number of subplots
num_plots = len(y_index)

# Create a figure with a grid of subplots
plt.figure(figsize=(4, 4 * num_plots))

# Loop through each index and create a subplot
for i in range(num_plots):
    plt.subplot(num_plots, 1, i + 1)  # (nrows, ncols, index)
    plt.boxplot([ng_lasso_list[i][0], ng_lasso_list[i][1]], tick_labels=['Lasso', 'Not selected'])
    plt.title(f'NG results of the selected OTU vs. non-selected OTUs by Lasso - {y_index[i]}')
    plt.ylabel('NG')
    plt.grid(axis='y')
# Adjust layout
plt.tight_layout()  # Adjusts the subplots to fit into the figure area.
plt.show()  # Show all plots at once

### Use LOOCV for fine tune model parameters and also the model's result

In [ ]:
targetLabel=burn_label
